In [1]:
# Importing libraries
import json
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import BartTokenizer, BartForConditionalGeneration

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console = Console(record=True)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
from torch import cuda
device = 'cuda:0' if cuda.is_available() else 'cpu'

In [10]:
model_path = 'facebook/bart-large_batch_8_lr_3e-060503-mix-with-eos/model_files'
tokenizer = BartTokenizer.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path)
model = model.to(device)

In [3]:
user_input = 'The Four Seasons'

In [33]:
def generate(user_input, model, tokenizer):
    prompt = "Generate keywords for the title: "
    placeholder = ". Keywords 1: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 2: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 3: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 4: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 5: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 6: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 7: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 8: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 9: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 10: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 11: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 12: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 13: ['<MASK>', '<MASK>', '<MASK>'] . Keywords 14: ['<MASK>', '<MASK>', '<MASK>'] </s>"
    bart_input = prompt + user_input + placeholder
    ids = tokenizer(bart_input, return_tensors="pt").input_ids.to(device)
    generated_ids = model.generate(
                  input_ids = ids,
                  max_length=512, 
                min_length = 200,
                  num_beams = 4,
                  no_repeat_ngram_size = 5,
                  #topp = 0.9,
                  #do_sample=True,
                  repetition_penalty=5.8, 
                  length_penalty=1, 
                  early_stopping=True
                  )
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    return preds

In [34]:
generate(user_input, model, tokenizer)

["Keywords 1: ['years', 'time', 'ago']. Keywords 2: ['life', 'happened', 'finally']. Keywords 3: ['day', 'woke','morning']. Keywords 4: ['room','sitting', 'bed']. Keywords 5: ['window', 'looked', 'backwards']. Keywords 6: ['sunrise','staring', 'darkness']. Keywords 7: ['night', 'couple', 'hours']. Keywords 8: ['doorway', 'heard', 'closer']. Keywords 9: ['Suddenly','sound', 'hear']. Keywords 10: ['screaming', 'loudly','started']. Keywords 11: ['house', 'ran', 'inside']. Keywords 12: ['stairs', 'quickly', 'opened']. Keywords 13: ['floor','shook', 'head']. Keywords 14: ['eyes', 'turned', 'face'] "]